In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
CLIENT_ID = 'XFDFOUQ0XNJJSLTZFN32VOIMA1N2OUG4K3HJHGN4NJ1HW4ZN' # your Foursquare ID
CLIENT_SECRET = 'NR10VHH3M4RBBAFJUD0EE2TO4XJ0Y5CTXY2J3BIMURWKECNA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: XFDFOUQ0XNJJSLTZFN32VOIMA1N2OUG4K3HJHGN4NJ1HW4ZN


In [7]:
import requests

request_parameters = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section": "coffee",
    "near": "Toronto",
    "radius": 1000,
    "limit": 50}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [8]:
d = data.json()["response"]
d.keys()

dict_keys(['geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'query', 'totalResults', 'suggestedBounds', 'groups'])

In [10]:
d["headerLocationGranularity"], d["headerLocation"], d["headerFullLocation"]

('city', 'Toronto', 'Toronto')

In [11]:
d["suggestedBounds"], d["totalResults"]

({'ne': {'lat': 43.70446656490709, 'lng': -79.40484434999999},
  'sw': {'lat': 43.70041378262348, 'lng': -79.42850865}},
 7)

In [12]:
d["geocode"]

{'what': '',
 'where': 'toronto',
 'center': {'lat': 43.70011, 'lng': -79.4163},
 'displayString': 'Toronto, ON, Canada',
 'cc': 'CA',
 'geometry': {'bounds': {'ne': {'lat': 43.855547992182615,
    'lng': -79.1040939249301},
   'sw': {'lat': 43.58464308210616, 'lng': -79.64745469478004}}},
 'slug': 'toronto',
 'longId': '72057594044095801'}

In [13]:
d["groups"][0].keys()

dict_keys(['type', 'name', 'items'])

In [14]:
d["groups"][0]["type"], d["groups"][0]["name"]

('Recommended Places', 'recommended')

In [15]:
items = d["groups"][0]["items"]
print("number of items: %i" % len(items))
items[0]

number of items: 7


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4b896ec4f964a520bd3532e3',
  'name': 'Hotel Gelato',
  'location': {'address': '532 Eglinton Ave. W',
   'crossStreet': 'at Heddington Ave.',
   'lat': 43.703477773135404,
   'lng': -79.41431071971614,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.703477773135404,
     'lng': -79.41431071971614}],
   'postalCode': 'M5N 1B4',
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['532 Eglinton Ave. W (at Heddington Ave.)',
    'Toronto ON M5N 1B4',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d16d941735',
    'name': 'Café',
    'pluralName': 'Cafés',
    'shortName': 'Café',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []},
  'venuePage': {'id': '

In [16]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    print(location)
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address =  ''
    if hasattr(location, 'address'):
      address = location['address']
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, postalcode, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "postalcode", "lat", "lng"])
print("found %i cafes" % len(df))
df.head()

{'address': '532 Eglinton Ave. W', 'crossStreet': 'at Heddington Ave.', 'lat': 43.703477773135404, 'lng': -79.41431071971614, 'labeledLatLngs': [{'label': 'display', 'lat': 43.703477773135404, 'lng': -79.41431071971614}], 'postalCode': 'M5N 1B4', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['532 Eglinton Ave. W (at Heddington Ave.)', 'Toronto ON M5N 1B4', 'Canada']}
{'address': '519 Eglinton Avenue West', 'crossStreet': 'Heddington Ave', 'lat': 43.70352947682181, 'lng': -79.4136982823359, 'labeledLatLngs': [{'label': 'display', 'lat': 43.70352947682181, 'lng': -79.4136982823359}], 'postalCode': 'M5N 1B1', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['519 Eglinton Avenue West (Heddington Ave)', 'Toronto ON M5N 1B1', 'Canada']}
{'address': '900 Eglinton Ave W', 'crossStreet': 'at Hilltop Rd.', 'lat': 43.700598, 'lng': -79.427433, 'labeledLatLngs': [{'label': 'display', 'lat': 43.700598, 'lng': -79.42743

,uid,name,shortname,address,postalcode,lat,lng
0,4b896ec4f964a520bd3532e3,Hotel Gelato,Café,,M5N 1B4,43.703478,-79.414311
1,4bdc79352a3a0f47a800b3b6,The Mad Bean Coffee House,Coffee Shop,,M5N 1B1,43.703529,-79.413698
2,4b0feea1f964a5206a6623e3,Starbucks,Coffee Shop,,M6C 2B6,43.700598,-79.427433
3,4ae99f04f964a52061b521e3,Starbucks,Coffee Shop,,M5N 1A5,43.704171,-79.411887
4,4b17ed23f964a520c1c923e3,Second Cup,Coffee Shop,,M5N 1A5,43.703583,-79.413824


In [17]:
Toronto_center = d["geocode"]["center"]
Toronto_center

{'lat': 43.70011, 'lng': -79.4163}

In [19]:
from folium import plugins


map_Toronto = folium.Map(location=[16.06778, 108.22083], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_Toronto)

add_markers(df)
hm_data = df[["lat", "lng"]].to_numpy().tolist()
map_Toronto.add_child(plugins.HeatMap(hm_data))

map_Toronto